In [ ]:
import requests
import pymysql
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from datetime import datetime, timedelta

import os
from dotenv import load_dotenv


# ✅ MySQL 연결 설정
DB_CONFIG = {
    "host": os.getenv("DB_HOST", "localhost"),
    "user": os.getenv("DB_USER", "crypto_user"),
    "password": os.getenv("DB_PASSWORD", "your_secure_password"),  # 환경 변수에서 불러오기
    "database": os.getenv("DB_NAME", "crypto_db"),
    "cursorclass": pymysql.cursors.DictCursor
}

# ✅ 코인 데이터 (KRW-TICKER 형식, `coin_id` 매핑)
coin_data = [
    (1, "KRW-XRP"),
    (2, "KRW-BTC"),
    (3, "KRW-ETH"),
    (4, "KRW-QTUM"),
    (5, "KRW-WAVES"),
    (6, "KRW-XEM"),
    (7, "KRW-ETC"),
    (8, "KRW-NEO"),
    (9, "KRW-SNT"),
    (10, "KRW-MTL"),
]

try:
    # ✅ MySQL 연결
    conn = pymysql.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # ✅ 기존 데이터 삭제 후 삽입 (중복 방지)
    cursor.execute("DELETE FROM coin;")
    sql = "INSERT INTO coin (coin_id, ticker) VALUES (%s, %s);"
    cursor.executemany(sql, [(cid, ticker) for cid, ticker in coin_data])

    conn.commit()
    print(f"✅ {cursor.rowcount} 개의 코인 데이터 삽입 완료.")
    
except pymysql.MySQLError as e:
    print(f"❌ 코인 삽입 오류: {e}")
finally:
    cursor.close()
    conn.close()

# ✅ CryptoPanic API 설정
# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
API_KEY = os.getenv("API_KEY")
API_URL = f"https://cryptopanic.com/api/v1/posts/?auth_token={API_KEY}"

# ✅ ChromeDriver 실행 (Selenium 사용)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# ✅ 날짜 파싱 함수 (UTC → KST 변환)
def parse_datetime(utc_str):
    try:
        dt = datetime.strptime(utc_str, "%Y-%m-%dT%H:%M:%SZ") + timedelta(hours=9)
        return dt.strftime("%Y-%m-%d %H:%M:%S")
    except Exception:
        return (datetime.utcnow() + timedelta(hours=9)).strftime("%Y-%m-%d %H:%M:%S")

# ✅ 뉴스 본문 가져오기 (Selenium 크롤링)
def get_news_content(news_url):
    try:
        driver.get(news_url)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        content_element = soup.select_one("div.description-body, div.article-content, div.entry-content, div.post-content, article")
        if content_element:
            article_text = "\n".join([p.get_text().strip() for p in content_element.find_all("p") if p.get_text().strip()])
        else:
            paragraphs = [p.get_text().strip() for p in soup.find_all("p") if len(p.get_text().strip()) > 50]
            article_text = "\n".join(paragraphs) if paragraphs else "본문 없음"

        if not article_text.strip() or len(article_text) < 50:
            return None, None

        translated_text = GoogleTranslator(source="en", target="ko").translate(article_text[:5000])
        return article_text, translated_text
    except Exception as e:
        print(f"❌ 본문 크롤링 오류: {news_url}, 오류 메시지: {e}")
        return None, None

# ✅ 뉴스 크롤링 & 저장
news_data = []
filtered_articles = 0

try:
    conn = pymysql.connect(**DB_CONFIG)
    cursor = conn.cursor()

    response = requests.get(API_URL, headers={"User-Agent": "Mozilla/5.0"})
    data = response.json()

    if "results" not in data:
        print(f"❌ API 응답 오류: {data}")
        exit()

    cursor.execute("SELECT ticker, coin_id FROM coin;")
    existing_coins = {row["ticker"]: row["coin_id"] for row in cursor.fetchall()}  # ✅ {ticker: coin_id}

    for post in data["results"]:
        if not post.get("currencies"):  # ✅ 티커가 없는 뉴스 필터링
            filtered_articles += 1
            continue

        title_en = post["title"]
        title_ko = GoogleTranslator(source="en", target="ko").translate(title_en)
        crypto_panic_url = post["url"]
        newspaper = post["source"]["title"] if "source" in post else "N/A"

        content_en, content_ko = get_news_content(crypto_panic_url)
        if not content_en:
            continue

        news_datetime = parse_datetime(post.get("published_at"))

        # ✅ 관련된 모든 코인 티커 가져오기
        related_coins = []
        for coin in post.get("currencies", []):
            ticker = f"KRW-{coin.get('code')}"  # ✅ KRW- 접두사 추가
            if len(ticker) < 5:  # ✅ 한 글자짜리 티커 필터링
                continue

            coin_id = existing_coins.get(ticker)
            if coin_id:
                related_coins.append(coin_id)

        if not related_coins:
            filtered_articles += 1
            continue

        # ✅ 각 코인별로 뉴스 저장 (coin_id가 None이 아닐 경우만 삽입)
        for coin_id in related_coins:
            if coin_id is None:
                print(f"⚠️ coin_id가 None이므로 저장되지 않음: {title_ko}")
                continue

            insert_news_query = """
            INSERT INTO news (newspaper, title, title_en, content, content_en, source, uploadtime, coin_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
            """
            cursor.execute(insert_news_query, (newspaper, title_ko, title_en, content_ko, content_en, crypto_panic_url, news_datetime, coin_id))
            conn.commit()

            print(f"✅ 저장된 뉴스 | 제목: {title_ko} | 코인 ID: {coin_id}")

    cursor.close()
    conn.close()

except Exception as e:
    print(f"❌ 크롤링 오류: {e}")

finally:
    driver.quit()

print(f"📌 티커가 없는 기사 {filtered_articles}개는 저장되지 않았습니다.")
